In [1]:
from datasets import load_dataset

ds = load_dataset("vanyacohen/CaT-Bench")

In [2]:
# print the first example and other dataset statistics
print(ds)

DatasetDict({
    train: Dataset({
        features: ['label', 'origin_text', 'destination_text', 'steps', 'title'],
        num_rows: 6934
    })
    validation: Dataset({
        features: ['label', 'origin_text', 'destination_text', 'steps', 'title'],
        num_rows: 808
    })
    test: Dataset({
        features: ['label', 'origin_text', 'destination_text', 'steps', 'title'],
        num_rows: 1420
    })
})


In [5]:
import pandas as pd
# Set the display options for word wrap
pd.set_option('display.max_colwidth', None)

In [41]:
# Your existing code
train_ds = ds['train']
train_example = train_ds[1]
print(train_example['label'])
print(train_example['origin_text'])
print(train_example['destination_text'])
print(train_example['steps'])
print(train_example['title'])

True
Bring to the boil and then transfer to slow cooker pouring over chicken.
Blend the onions, garlic, ginger and chilli in a blender or liquidiser, or chop very finely.
['Blend the onions, garlic, ginger and chilli in a blender or liquidiser, or chop very finely.', 'Cut each thigh into 4 pieces, heat the oil in a large frying pan and add the chicken.', 'Cook over a high heat until evenly browned.', 'Remove from pan and place in slow cooker.', 'Add the butter to the pan and when melted add the onion mixture.', 'Cook over moderate heat for 3 minutes.', 'Add all the spices and cook for a further minute.', 'Mix in chicken stock, sugar and salt.', 'Bring to the boil and then transfer to slow cooker pouring over chicken.', 'Cover and cook for 5 - 7 hours on Low (or 3 hours on high).', 'Add the cream and ground up nuts.', 'Serve with rice and naan bread.']
Slow_cooked_chicken_korma_recipe_-_All_recipes_UK


In [42]:
def format_input_for_prompt(example, temporal_relation="before"):
    title = example['title']
    origin_text = example['origin_text']
    destination_text = example['destination_text']
    steps = example['steps']
    enumerated_steps = list(enumerate(steps, start=1))
    prompt_steps = [f"{step[0]}. {step[1]}" for step in enumerated_steps]
    prompt_steps_string = '\n'.join(prompt_steps)
    i = next((step[0] for step in enumerated_steps if step[1] == origin_text), None)
    j = next((step[0] for step in enumerated_steps if step[1] == destination_text), None)
    if i is None or j is None:
        raise ValueError("Origin or destination text not found in steps")
    if temporal_relation is None:
        if i < j:
            temporal_relation = "before"
        else:
            temporal_relation = "after"
    
    return title, prompt_steps_string, i, j, temporal_relation

In [43]:
title, prompt_steps_string, i, j, temporal_relation = format_input_for_prompt(train_example)

In [44]:
print(prompt_steps_string)

1. Blend the onions, garlic, ginger and chilli in a blender or liquidiser, or chop very finely.
2. Cut each thigh into 4 pieces, heat the oil in a large frying pan and add the chicken.
3. Cook over a high heat until evenly browned.
4. Remove from pan and place in slow cooker.
5. Add the butter to the pan and when melted add the onion mixture.
6. Cook over moderate heat for 3 minutes.
7. Add all the spices and cook for a further minute.
8. Mix in chicken stock, sugar and salt.
9. Bring to the boil and then transfer to slow cooker pouring over chicken.
10. Cover and cook for 5 - 7 hours on Low (or 3 hours on high).
11. Add the cream and ground up nuts.
12. Serve with rice and naan bread.


In [45]:
def get_answer_only_prompt(title, procedure, i, j, temporal_relation="before"):
    TASK = "Given a goal, a procedure to achieve that goal and a question about the steps in the procedure, you are required to answer the question in one sentence."
    GOAL = "Goal: " + title
    PROCEDURE = "Procedure: \n" + procedure
    QUERY = f"Must Step {i} happen {temporal_relation} Step {j}? Select between yes or no."
    
    full_prompt = f"{TASK}\n\n{GOAL}\n\n{PROCEDURE}\n\n{QUERY}"
    return full_prompt
    
def get_answer_explanation_prompt(title, procedure, i, j, temporal_relation="before"):
    TASK = "Given a goal, a procedure to achieve that goal and a question about the steps in the procedure, you are required to answer the question in one sentence."
    GOAL = "Goal: " + title
    PROCEDURE = "Procedure: \n" + procedure
    QUERY_1 = f"1. Must Step {i} happen {temporal_relation} Step {j}? Select between yes or no."
    QUERY_2 = "Explain why or why not."
    FORMAT = "Format your answer as JSON with the key value pairs \"binary_answer\": \"yes/no answer to Q1\"; \"why_answer\": \"answer to Q2\""
    
    full_prompt = f"{TASK}\n\n{GOAL}\n\n{PROCEDURE}\n\n{QUERY_1}\n{QUERY_2}\n\n{FORMAT}"
    return full_prompt

def get_explanation_answer_prompt(title, procedure, i, j, temporal_relation="before"):
    TASK = "Given a goal, a procedure to achieve that goal and a question about the steps in the procedure, you are required to answer the question in one sentence."
    GOAL = "Goal: " + title
    PROCEDURE = "Procedure: \n" + procedure
    QUERY_1 = f"1. Explain why or why not Step {i} must happen {temporal_relation} Step {j}. Think step by step."
    QUERY_2 = f"2. Must Step {i} happen {temporal_relation} Step {j}? Select between yes or no"
    FORMAT = "Format your answer as JSON with the key value pairs \"why_answer\": \"answer to Q1\"; \"binary_answer\": \"yes/no answer to Q2\""
    
    full_prompt = f"{TASK}\n\n{GOAL}\n\n{PROCEDURE}\n\n{QUERY_1}\n{QUERY_2}\n\n{FORMAT}"
    return full_prompt

def get_answer_explanation_prompt(title, procedure, i, j, temporal_relation="before"):
    TASK = "Given a goal, a procedure to achieve that goal and a question about the steps in the procedure, you are required to answer the question in one sentence."
    GOAL = "Goal: " + title
    PROCEDURE = "Procedure: \n" + procedure
    QUERY_1 = f"1. Must Step {i} happen {temporal_relation} Step {j}? Select between yes or no."
    QUERY_2 = "Explain why or why not."
    FORMAT = "Format your answer as follows: \n\n Answer 1: yes/no \nAnswer 2: your answer in one sentence"
    
    full_prompt = f"{TASK}\n\n{GOAL}\n\n{PROCEDURE}\n\n{QUERY_1}\n{QUERY_2}\n\n{FORMAT}"
    return full_prompt


In [46]:
print(get_answer_only_prompt(title, prompt_steps_string, i, j, temporal_relation))

Given a goal, a procedure to achieve that goal and a question about the steps in the procedure, you are required to answer the question in one sentence.

Goal: Slow_cooked_chicken_korma_recipe_-_All_recipes_UK

Procedure: 
1. Blend the onions, garlic, ginger and chilli in a blender or liquidiser, or chop very finely.
2. Cut each thigh into 4 pieces, heat the oil in a large frying pan and add the chicken.
3. Cook over a high heat until evenly browned.
4. Remove from pan and place in slow cooker.
5. Add the butter to the pan and when melted add the onion mixture.
6. Cook over moderate heat for 3 minutes.
7. Add all the spices and cook for a further minute.
8. Mix in chicken stock, sugar and salt.
9. Bring to the boil and then transfer to slow cooker pouring over chicken.
10. Cover and cook for 5 - 7 hours on Low (or 3 hours on high).
11. Add the cream and ground up nuts.
12. Serve with rice and naan bread.

Must Step 9 happen before Step 1? Select between yes or no.


In [10]:
import os
STORAGE_DIR = os.path.join(os.getcwd(), "../data_storage/cat_bench")
binary_label_path = os.path.join(STORAGE_DIR, "binary_label")
train_csv_path = os.path.join(binary_label_path, "train.csv")

In [11]:
import pandas as pd
df_train_binary = pd.read_csv(train_csv_path)
df_train_binary.head()

,index,recipe_name,sentence1,sentence2,label
0,0,Pico_de_gallo_chicken_quesadillas_recipe_-_All...,"In a clean, heavy frying pan, heat one flour t...",Set aside.,1
1,1,Slow_cooked_chicken_korma_recipe_-_All_recipes_UK,Bring to the boil and then transfer to slow co...,"Blend the onions, garlic, ginger and chilli in...",1
2,2,vodka-penne,Season finished sauce with salt and pepper.,Heat the olive oil in a large saucepan over me...,1
3,3,dal-fritters,"These will absorb the oil in the pan, but that...",Combine all the ingredients together.,1
4,4,cardiganshire-cawl,"Add potatoes, and simmer until tender, 15 to 2...",Simmer for 1 hour.,1


In [14]:
df_train_binary.iloc[0]['sentence1'] + " " + df_train_binary.iloc[0]['sentence2']

'In a clean, heavy frying pan, heat one flour tortilla. Set aside.'